## Presentation Type Classifier and Networks

#### Network Visualization with Louvain Communities

* Read more about Louvain Community Detection Algorithms:  https://towardsdatascience.com/community-detection-algorithms-9bd8951e7dae

* **Updated June 2022**
* **Key Features**:

    * Uses `getDistance` to identify `close matches` with side-by-side comparison of soggetti.  
    * For example: With a distance of "1", the soggetti `4, 1, 2, 3`, and `5, 1, 2, 3` will count as the same. These are reported as "flexed entries" in a separate column.
    * Finds time intervals between entries (expressed as offsets, like `8.0, 4.0, 8.0`)
    * Finds melodic intervals between first note of successive entries in each pattern (like `P-5, P-8`)
    * Labels Fuga, PEn, and ID according to time intervals between entries. NIm not yet supported!
    * If two entries are separated by more than 10 bars (80 offsets), the tool resets to a new pattern
    * Counts number of entries
    * Provides offset and measure/beat locations
    * Sorts all presentation types by the order in which they appear in the piece
    * Reports voice names of the entries, in order of their appearance
    * Omits singleton soggetti (just one entry of a given motive in isolation)
    
* Read the documentation:  `print(piece.presentationTypes.__doc__)`

In [17]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy as np
import itertools
from itertools import combinations
import networkx as nx
from community import community_louvain
from copy import deepcopy
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


## A.  Start by Finding Presentation Types in a Corpus of Pieces 

- Results are combined into a single dataframe
- Option to save combined results as CSV

In [29]:
prefix = 'https://crimproject.org/mei/'

piece_list = ['CRIM_Model_0019',
             'CRIM_Mass_0019_1',
             'CRIM_Mass_0019_2',
             'CRIM_Mass_0019_3',
             'CRIM_Mass_0019_4']
#              'CRIM_Mass_0019_5']

suffix = '.mei'

In [30]:

final = pd.DataFrame()

for piece in piece_list:
    file_url = prefix + piece 
    url = prefix + piece + suffix
    piece = importScore(url)   
    output = piece.presentationTypes(limit_to_entries = True,
                                      edit_distance_threshold = 0,
                                    include_hidden_types = True,
                                    combine_unisons = True,
                                   melodic_ngram_length = 4)

    final['Validation'] = ""
    final['Comments'] = ""
    final = final.append(output, ignore_index=True)
final.head()
# final.to_csv("corpus_classified.csv")
# final.to_csv("saved_csv/CRIM_Mass_0019_P_Types_hidden.csv")

Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.


,Validation,Comments,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0]","[P-5, P-4, P-5]","[0.0, 8.0, 40.0, 48.0]","[(-3, 3, 2, -2)]","[8.0, 32.0, 8.0]","[Cantus, Altus, Tenor, Bassus]",ID,4.0,False
1,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 11/3.0, 14/1.0, 1...","[P-5, P-4, P-5, P8, P5, P-8]","[0.0, 8.0, 40.0, 48.0, 84.0, 104.0, 128.0]","[(-3, 3, 2, -2)]","[8.0, 32.0, 8.0, 36.0, 20.0, 24.0]","[Cantus, Altus, Tenor, Bassus, Altus, Cantus, ...",FUGA,7.0,False
2,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,40.0,"[6/1.0, 11/3.0, 17/1.0]","[P4, P-4]","[40.0, 84.0, 128.0]","[(-3, 3, 2, -2)]","[44.0, 44.0]","[Tenor, Altus, Bassus]",PEN,3.0,False
3,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,150.0,"[19/4.0, 21/4.0, 22/4.0, 25/4.0, 27/4.0, 29/4....","[P8, P-5, P-4, P-5, P8, P8, P-11]","[150.0, 166.0, 174.0, 198.0, 214.0, 230.0, 250...","[(-3, 2, 2, -2)]","[16.0, 8.0, 24.0, 16.0, 16.0, 20.0, 24.0]","[Tenor, Cantus, Altus, Tenor, Bassus, Altus, C...",FUGA,8.0,False
4,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,150.0,"[19/4.0, 22/4.0, 32/2.0, 35/2.0]","[P4, P8, P-11]","[150.0, 174.0, 250.0, 274.0]","[(-3, 2, 2, -2)]","[24.0, 76.0, 24.0]","[Tenor, Altus, Cantus, Bassus]",ID,4.0,False


In [31]:
# filter for only ID or PEN
pen_filter = final['Presentation_Type'].str.contains("PEN")
pens_only = final[pen_filter]
final = pens_only.copy()
final


,Validation,Comments,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
2,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,40.0,"[6/1.0, 11/3.0, 17/1.0]","[P4, P-4]","[40.0, 84.0, 128.0]","[(-3, 3, 2, -2)]","[44.0, 44.0]","[Tenor, Altus, Bassus]",PEN,3.0,False
7,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,150.0,"[19/4.0, 22/4.0, 25/4.0]","[P4, P-4]","[150.0, 174.0, 198.0]","[(-3, 2, 2, -2)]","[24.0, 24.0]","[Tenor, Altus, Tenor]",PEN,3.0,False
8,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,166.0,"[21/4.0, 25/4.0, 29/4.0]","[P-8, P4]","[166.0, 198.0, 230.0]","[(-3, 2, 2, -2)]","[32.0, 32.0]","[Cantus, Tenor, Altus]",PEN,3.0,False
10,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,198.0,"[25/4.0, 27/4.0, 29/4.0]","[P-5, P8]","[198.0, 214.0, 230.0]","[(-3, 2, 2, -2)]","[16.0, 16.0]","[Tenor, Bassus, Altus]",PEN,3.0,False
12,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,298.0,"[38/2.0, 42/2.0, 46/2.0]","[M-9, P15]","[298.0, 330.0, 362.0]","[(-3, 3, -2, -2)]","[32.0, 32.0]","[Altus, Bassus, Cantus]",PEN,3.0,False
16,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,394.0,"[50/2.0, 51/4.0, 53/2.0]","[P5, P-8]","[394.0, 406.0, 418.0]","[(-2, 2, 3, -2)]","[12.0, 12.0]","[Altus, Cantus, Bassus]",PEN,3.0,False
17,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,406.0,"[51/4.0, 56/4.0, 61/4.0]","[P1, P1]","[406.0, 446.0, 486.0]","[(-2, 2, 3, -2)]","[40.0, 40.0]","[Cantus, Altus, Cantus]",PEN,3.0,False
27,,,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,372.0,"[45/3.0, 47/3.0, 49/3.0]","[P8, P-8]","[372.0, 388.0, 404.0]","[(-3, 3, -2, -2)]","[16.0, 16.0]","[Tenor, Cantus, Bassus]",PEN,3.0,False
28,,,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,372.0,"[45/3.0, 49/3.0, 53/3.0]","[P1, P-5]","[372.0, 404.0, 436.0]","[(-3, 3, -2, -2)]","[32.0, 32.0]","[Tenor, Bassus, Bassus]",PEN,3.0,False
35,,,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Gloria,274.0,"[35/2.0, 37/4.0, 40/2.0]","[P-12, P12]","[274.0, 294.0, 314.0]","[(-2, 2, 3, -2)]","[20.0, 20.0]","[Cantus, Bassus, Cantus]",PEN,3.0,False


In [32]:
len(final)

47

In [33]:
# filter for maximum time interval between entries
final['pen_increment'] = final['Time_Entry_Intervals'].apply(set).apply(sum)
final = final[final['pen_increment'] < 40].copy()


In [34]:
# remove a particular piece
# final = final[final['Title'] != 'Veni sponsa Christi'].copy()


## B. Network Visualization with Louvain Communities

* This will only make sense for a corpus!

## Add Combined Data for Melodic, Time and Soggetti as "Words" for Network Edges

* We combine the lists of time and melodic intervals of entries as strings (thus a kind of word that uniquely identifies that pattern)
* We do the same for soggetti, combining the melodic intervals themselves as a string
* These are in turn assembled in various combinations:  TINT, MINT, and SOG, also as strings

In [35]:
final["MINT"] = final["Melodic_Entry_Intervals"].apply(joiner)
final["TINT"] = final["Time_Entry_Intervals"].apply(joiner)

final['SOG'] = final['Soggetti'].apply(clean_melody_new)
final['ALL'] = final["MINT"] + '_' + final["TINT"] + '_' + final['SOG']

final["ALL_INT"] = final["MINT"] + '_' + final["TINT"]
final["MINT_SOG"] = final["MINT"] + '_' + final["SOG"]
final["TINT_SOG"] = final["TINT"] + '_' + final["SOG"]
final.head()

,Validation,Comments,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,...,Number_Entries,Flexed_Entries,pen_increment,MINT,TINT,SOG,ALL,ALL_INT,MINT_SOG,TINT_SOG
7,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,150.0,"[19/4.0, 22/4.0, 25/4.0]","[P4, P-4]","[150.0, 174.0, 198.0]","[(-3, 2, 2, -2)]","[24.0, 24.0]",...,3.0,False,24.0,P4_P-4,24.0_24.0,-3_2_2_-2,P4_P-4_24.0_24.0_-3_2_2_-2,P4_P-4_24.0_24.0,P4_P-4_-3_2_2_-2,24.0_24.0_-3_2_2_-2
8,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,166.0,"[21/4.0, 25/4.0, 29/4.0]","[P-8, P4]","[166.0, 198.0, 230.0]","[(-3, 2, 2, -2)]","[32.0, 32.0]",...,3.0,False,32.0,P-8_P4,32.0_32.0,-3_2_2_-2,P-8_P4_32.0_32.0_-3_2_2_-2,P-8_P4_32.0_32.0,P-8_P4_-3_2_2_-2,32.0_32.0_-3_2_2_-2
10,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,198.0,"[25/4.0, 27/4.0, 29/4.0]","[P-5, P8]","[198.0, 214.0, 230.0]","[(-3, 2, 2, -2)]","[16.0, 16.0]",...,3.0,False,16.0,P-5_P8,16.0_16.0,-3_2_2_-2,P-5_P8_16.0_16.0_-3_2_2_-2,P-5_P8_16.0_16.0,P-5_P8_-3_2_2_-2,16.0_16.0_-3_2_2_-2
12,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,298.0,"[38/2.0, 42/2.0, 46/2.0]","[M-9, P15]","[298.0, 330.0, 362.0]","[(-3, 3, -2, -2)]","[32.0, 32.0]",...,3.0,False,32.0,M-9_P15,32.0_32.0,-3_3_-2_-2,M-9_P15_32.0_32.0_-3_3_-2_-2,M-9_P15_32.0_32.0,M-9_P15_-3_3_-2_-2,32.0_32.0_-3_3_-2_-2
16,,,Giovanni Pierluigi da Palestrina,Veni sponsa Christi,394.0,"[50/2.0, 51/4.0, 53/2.0]","[P5, P-8]","[394.0, 406.0, 418.0]","[(-2, 2, 3, -2)]","[12.0, 12.0]",...,3.0,False,12.0,P5_P-8,12.0_12.0,-2_2_3_-2,P5_P-8_12.0_12.0_-2_2_3_-2,P5_P-8_12.0_12.0,P5_P-8_-2_2_3_-2,12.0_12.0_-2_2_3_-2


### Make Groups, Based on Selected Criteria:

* MINT
* TINT
* SOG
* Or any of the combinations:
    * ALL (MINT + TINT + Soggetti)
    * ALL_INT (MINT + TINT)
    * MINT_SOG (MINT + Soggetti)
    * TINT_SOG (TINT + Soggetti)

In [40]:

output_grouped = final.groupby('Title')['SOG'].apply(list).reset_index()
output_grouped


,Title,SOG
0,Missa Veni sponsa Christi: Credo,"[-2_-2_-2_-2, 4_2_2_2, 4_2_2_2, 4_2_2_2, 4_2_2..."
1,Missa Veni sponsa Christi: Gloria,"[-2_2_3_-2, -3_3_2_-2, -3_3_2_-2, -3_3_2_-2, -..."
2,Missa Veni sponsa Christi: Kyrie,"[-3_3_-2_-2, -3_3_-2_-2]"
3,Missa Veni sponsa Christi: Sanctus,"[-2_2_-3_2, -3_3_2_-2, -2_2_3_-2, -2_2_3_-2, -..."
4,Veni sponsa Christi,"[-3_2_2_-2, -3_2_2_-2, -3_2_2_-2, -3_3_-2_-2, ..."


### Group and Find Unique Pairs

- the pairing must be applied to columns that have more than one value, like Title, since these form the edges

In [42]:
pairs = output_grouped.SOG.apply(lambda x: list(combinations(x, 2)))
pairs2 = pairs.explode().dropna()
unique_pairs = pairs.explode().dropna().unique()

In [43]:
unique_pairs

array([('-2_-2_-2_-2', '4_2_2_2'), ('4_2_2_2', '4_2_2_2'),
       ('-2_2_3_-2', '-3_3_2_-2'), ('-2_2_3_-2', '-2_2_3_-2'),
       ('-3_3_2_-2', '-3_3_2_-2'), ('-3_3_2_-2', '-2_2_3_-2'),
       ('-3_3_-2_-2', '-3_3_-2_-2'), ('-2_2_-3_2', '-3_3_2_-2'),
       ('-2_2_-3_2', '-2_2_3_-2'), ('-2_2_-3_2', '-3_2_2_2'),
       ('-2_2_-3_2', '-3_2_2_-2'), ('-3_3_2_-2', '-3_2_2_2'),
       ('-3_3_2_-2', '-3_2_2_-2'), ('-2_2_3_-2', '-3_2_2_2'),
       ('-2_2_3_-2', '-3_2_2_-2'), ('-3_2_2_2', '-3_2_2_2'),
       ('-3_2_2_2', '-3_3_2_-2'), ('-3_2_2_2', '-3_2_2_-2'),
       ('-3_2_2_-2', '-3_2_2_-2'), ('-3_2_2_-2', '-3_3_-2_-2'),
       ('-3_2_2_-2', '-2_2_3_-2'), ('-3_3_-2_-2', '-2_2_3_-2')],
      dtype=object)

In [44]:
pd.Series(unique_pairs).isna().sum()

0

In [45]:
def add_communities(G):
    G = deepcopy(G)
    partition = community_louvain.best_partition(G)
    nx.set_node_attributes(G, partition, "group")
    return G
pyvis_graph = Network(notebook=False, width="1800", height="1400", bgcolor="white", font_color="black")
G = nx.Graph()
G.add_edges_from(unique_pairs)
G = add_communities(G)
pyvis_graph.from_nx(G)
pyvis_graph.show('Mass_SOG_MINT.html')